#   Homework 3
## Sentiment analysis using Neural Networks

Total: 50 Points


In this homework we will perform sentiment analysis using a few simple Neural Network based architectures.
For this problem we use the IMDB Large Movie Review Dataset. The dataset contains 25,000 highly polar movie reviews for both train and test dataset, each with 12,500 positive (greater than equal to 7/10 rating) and 12,500 negative reviews(less than equal to 4/10 rating). 

Use "https://keras.io/" for keras documentation. Please use Python 3. GPU is not required but it will help improve the training speed for each problem.

Please save the notebook with your cell outputs. You will not be graded if your outputs are not present below the homework cell. Also note your outputs will be unique since you will be using your the last numbers of your uni as your random seed (In the third cell). Make sure you submit this iPython file, with the saved outputs. The submission format must be 'hw3/hw3.ipynb'. You will not submit any other files. If you do save your model weights, you will not submit them. You will however, make sure your model weights do get saved in the 'weights' folder and can be retrieved from there as well.

Please fill your details below.



Name: Sicong Fang

Uni: sf2796

Email: sf2796@columbia.edu


In [3]:
from os import listdir
import random
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Dense, Dropout, Reshape, Merge, BatchNormalization, TimeDistributed, Lambda, Activation, LSTM, Flatten, Convolution1D, GRU, MaxPooling1D
from keras.regularizers import l2
from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping
#from keras import initializers
from keras import backend as K
from keras.optimizers import SGD
from keras.optimizers import Adadelta
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras import optimizers
import numpy as np

Using TensorFlow backend.


In [4]:


#we retrieve train and test file names

train_dir = "./aclImdb/train/"
test_dir = "./aclImdb/test/"
tr_review = [re_filename for re_filename in listdir(train_dir)]
te_review = [re_filename for re_filename in listdir(test_dir)]

#we initialize the train and test arrays

tr_X = []
tr_Y = []
te_X = []
te_Y = []

#we arrange the reviews into the train and test arrays 

for review_file in tr_review:
    f_review = open(train_dir+review_file, "r")
    str_review = f_review.readline()
    str_review = " ".join(str_review.split(' '))
    tr_X.append(str_review)
    y_truth = int (review_file.split('.')[0].split('_')[1])
    if y_truth>=7:
        tr_Y.append(1)
    else:
        tr_Y.append(0)
        
for review_file in te_review:
    f_review = open(test_dir+review_file, "r")
    str_review = f_review.readline()
    str_review = " ".join(str_review.split(' '))
    te_X.append(str_review)
    y_truth = int (review_file.split('.')[0].split('_')[1])
    if y_truth>=7:
        te_Y.append(1)
    else:
        te_Y.append(0)
        

We will now create the validation set from the train set

use the last 4 numbers of your uni for the seed value seed to ensure all answers remain unique.

In [5]:
#replace 2 (SEED) with the last 4 numbers of your Uni
#Uni: 2796
SEED = 2796
seed_counter = 0
while(1):

    shuffle_combine = list(zip(tr_X, tr_Y))
    random.seed(SEED+seed_counter)
    seed_counter+=1
    random.shuffle(shuffle_combine)

    tr_X, tr_Y = zip(*shuffle_combine)

    val_X = tr_X[:5000]
    val_Y = tr_Y[:5000]

    counter = 0
    for label in val_Y:
        counter+=label

    print (counter)
    print (seed_counter)
    if(counter>2400 and counter <2600):
        tr_X = tr_X[5000:]
        tr_Y = tr_Y[5000:]
        break

2507
1


In [6]:


print("Length of Train review set : " + str(len(tr_X)))
print("Length of Train label set : " + str(len(tr_Y)))
print("Length of Validation review set : " + str(len(val_X)))
print("Length of Validation label set : " + str(len(val_Y)))
print("Length of Test review set : " + str(len(te_X)))
print("Length of Test label set : " + str(len(te_Y)))
print("*****************************************")
print("Some sample Reviews Train sets and their labels")
print(tr_X[0][:150])
print(tr_Y[0])
print(tr_X[1][:150])
print(tr_Y[1])
print(tr_X[2][:150])
print(tr_Y[2])
print(tr_X[3][:150])
print(tr_Y[3])
print(tr_X[4][:150])
print(tr_Y[4])

Length of Train review set : 20000
Length of Train label set : 20000
Length of Validation review set : 5000
Length of Validation label set : 5000
Length of Test review set : 25000
Length of Test label set : 25000
*****************************************
Some sample Reviews Train sets and their labels
As a huge fan of only the first 2 seasons of BSG and the stand alone feature BSG Razor I was hoping that this release would return the franchise to it
0
Having heard quite positive reviews and having seen the trailer I had to see this movie. With William H. Macy, Luis Guzman, Michael Jeter and Sam Rock
1
Sure, Titanic was a good movie, the first time you see it, but you really should see it a second time and your opinion of the film will definetly chan
1
I saw this when on The Wonderful World of Disney as a kid, so I didn't recall much of it. As I watched it recently, I sat there thinking, "This is the
1
Frustrating to watch because of one man's stubbornness to leave his native country fo

In [7]:
#we collect all the reviews from train validation and test set to generate 
texts = []
texts += tr_X 
texts += te_X 
texts += val_X
len(texts)



#we clip the sentence length to first 250 words. 
MAX_SEQUENCE_LENGTH = 250

#length of vocab, Tokenizer will only use vocab_len most common words
vocab_len = 25000

#we tokenize the texts and convert all the words to tokens
tokenizer = Tokenizer(num_words=vocab_len)
tokenizer.fit_on_texts(texts)

token_tr_X = tokenizer.texts_to_sequences(tr_X)
token_te_X = tokenizer.texts_to_sequences(te_X)
token_val_X = tokenizer.texts_to_sequences(val_X)

#to ensure all reviews have the same length, we pad the smaller reviews with 0, 
#and cut the larger reviews to a max length 
#(we clip from the top, as the end of the reviews generally have a conclusion which provides better features)
x_train = sequence.pad_sequences(token_tr_X, maxlen=MAX_SEQUENCE_LENGTH)
x_test = sequence.pad_sequences(token_te_X, maxlen=MAX_SEQUENCE_LENGTH)
x_val = sequence.pad_sequences(token_val_X, maxlen=MAX_SEQUENCE_LENGTH)


#changes the labels to one-hot encoding
y_train = np_utils.to_categorical(tr_Y)
y_test = np_utils.to_categorical(te_Y)
y_val = np_utils.to_categorical(val_Y)


In [8]:
print('X_train shape:', x_train.shape)
print('X_test shape:', x_test.shape)
print('X_val shape:', x_val.shape)

print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)
print('y_val shape:', y_val.shape)


print("*****************************************")
print("Tokenized Reviews Train sets and their labels")
print(x_train[0][:20])
print(y_train[0])
print()
print(x_train[1][:20])
print(y_train[1])
print()
print(x_train[2][:20])
print(y_train[2])
print()
print(x_train[3][:20])
print(y_train[3])
print()
print(x_train[4][:20])
print(y_train[4])
print()

X_train shape: (20000, 250)
X_test shape: (25000, 250)
X_val shape: (5000, 250)
y_train shape: (20000, 2)
y_test shape: (25000, 2)
y_val shape: (5000, 2)
*****************************************
Tokenized Reviews Train sets and their labels
[    6   666   727  2130    14  3336     5     3  1076 10755     4   310
   286    43     1   227     6  2531    16    47]
[ 1.  0.]

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[ 0.  1.]

[   0    0    0    0  247 3890   13    3   49   17    1   86   55   22   63
    9   18   22   62  142]
[ 0.  1.]

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[ 0.  1.]

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[ 0.  1.]



********************************************

As you can see the reviews have now been transformed into indices to tokenized vocabulary and the labels have been converted to one-hot encoding. We can now go ahead and feed these sequences to Neural Network Models.

********************************************

# Part A

Building your first model (5 Points)

Construct this sequential model using Keras :

![title](img/model1.jpg)

In [7]:
print('Build model...')

## implement model here

model = Sequential()
model.add(Embedding(vocab_len, 128, input_length=MAX_SEQUENCE_LENGTH))
model.add(Flatten())
model.add(Dense(200, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

## compille it here according to instructions

model.summary()

print("Model Built")

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 250, 128)          3200000   
_________________________________________________________________
flatten_1 (Flatten)          (None, 32000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 200)               6400200   
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 402       
Total params: 9,600,602
Trainable params: 9,600,602
Non-trainable params: 0
_________________________________________________________________
Model Built


In [11]:
print('Train...')
model.fit(x_train, y_train,
          batch_size=32,
          epochs=4,
          validation_data=(x_val, y_val),
          verbose = 1,
         shuffle = True)

Train...
Train on 20000 samples, validate on 5000 samples
Epoch 1/4
20000/20000 [==============================] - 48s 2ms/step - loss: 1.2090e-06 - acc: 1.0000 - val_loss: 0.7556 - val_acc: 0.8608
Epoch 2/4
20000/20000 [==============================] - 48s 2ms/step - loss: 8.7715e-07 - acc: 1.0000 - val_loss: 0.7688 - val_acc: 0.8608
Epoch 3/4
20000/20000 [==============================] - 49s 2ms/step - loss: 6.4541e-07 - acc: 1.0000 - val_loss: 0.7824 - val_acc: 0.8610
Epoch 4/4
20000/20000 [==============================] - 49s 2ms/step - loss: 4.8273e-07 - acc: 1.0000 - val_loss: 0.7961 - val_acc: 0.8608


# Part B

Stacking Fully Connected Layers (5 points)

Construct this sequential model using Keras :

![title](img/model2.jpg)

In [7]:
print('Build model...')

## implement model here

model = Sequential()
model.add(Embedding(vocab_len, 128, input_length=MAX_SEQUENCE_LENGTH))
model.add(Flatten())
model.add(Dense(200, activation='relu'))
model.add(Dense(200, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
## compille it here according to instructions

model.summary()

print("Model Built")

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 250, 128)          3200000   
_________________________________________________________________
flatten_1 (Flatten)          (None, 32000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 200)               6400200   
_________________________________________________________________
dense_2 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 402       
Total params: 9,640,802
Trainable params: 9,640,802
Non-trainable params: 0
_________________________________________________________________
Model Built


In [8]:
print('Train...')
model.fit(x_train, y_train,
          batch_size=32,
          epochs=4,
          validation_data=(x_val, y_val),
          verbose = 1,
         shuffle = True)

Train...
Train on 20000 samples, validate on 5000 samples
Epoch 1/4
20000/20000 [==============================] - 49s 2ms/step - loss: 0.3954 - acc: 0.8071 - val_loss: 0.3134 - val_acc: 0.8634
Epoch 2/4
20000/20000 [==============================] - 49s 2ms/step - loss: 0.0050 - acc: 0.9985 - val_loss: 0.7362 - val_acc: 0.8468
Epoch 4/4
20000/20000 [==============================] - 49s 2ms/step - loss: 0.0101 - acc: 0.9970 - val_loss: 0.8976 - val_acc: 0.8304


# Part C

Using LSTMS based networks(5 Points) 

Construct this sequential model using Keras :

![title](img/model3.jpg)

In [7]:
print('Build model...')

## implement model here

model = Sequential()
model.add(Embedding(vocab_len, 128, input_length=MAX_SEQUENCE_LENGTH))
model.add(LSTM(128))
model.add(Dense(200, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
## compille it here according to instructions

model.summary()

print("Model Built")

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 250, 128)          3200000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_1 (Dense)              (None, 200)               25800     
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 402       
Total params: 3,357,786
Trainable params: 3,357,786
Non-trainable params: 0
_________________________________________________________________
Model Built


In [8]:

print('Train...')
model.fit(x_train, y_train,
          batch_size=32,
          epochs=5,
          validation_data=(x_val, y_val),
          verbose = 1,
         shuffle = True)

Train...
Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 168s 8ms/step - loss: 0.4141 - acc: 0.8055 - val_loss: 0.4769 - val_acc: 0.7738
Epoch 2/5
20000/20000 [==============================] - 166s 8ms/step - loss: 0.2137 - acc: 0.9177 - val_loss: 0.4150 - val_acc: 0.8190
Epoch 3/5
20000/20000 [==============================] - 166s 8ms/step - loss: 0.1116 - acc: 0.9594 - val_loss: 0.4659 - val_acc: 0.8644
Epoch 4/5
20000/20000 [==============================] - 166s 8ms/step - loss: 0.0776 - acc: 0.9720 - val_loss: 0.4777 - val_acc: 0.8228
Epoch 5/5
20000/20000 [==============================] - 165s 8ms/step - loss: 0.0442 - acc: 0.9856 - val_loss: 0.6475 - val_acc: 0.8412


# Part D

Adding Pretrained Word Embeddings(10 Points)

Construct this sequential model using Keras :

Correction: The Embedding Layer Dimension (1st box) is 300, not 128.

![title](img/model4.jpg)

In [10]:
import codecs

#dimension of Glove Embeddings.
EMBEDDING_DIM = 300

word_index = tokenizer.word_index
print('Found %s unique tokens' % len(word_index))

#load glove embeddings
gembeddings_index = {}
with codecs.open('glove.42B.300d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        gembedding = np.asarray(values[1:], dtype='float32')
        gembeddings_index[word] = gembedding
#
f.close()
print('G Word embeddings:', len(gembeddings_index))

# nb_words contains the total length of vocab
nb_words = len(word_index) +1

#get glove embeddings for each word in tokenizer.
#g_word_embedding_matrix holds the embeddings dictionary
g_word_embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))

for word, i in word_index.items():
    gembedding_vector = gembeddings_index.get(word)
    if gembedding_vector is not None:
        g_word_embedding_matrix[i] = gembedding_vector
        
#total words in the tokenizer not in Embedding matrix
print('G Null word embeddings: %d' % np.sum(np.sum(g_word_embedding_matrix, axis=1) == 0))



Found 124252 unique tokens
G Word embeddings: 1917494
G Null word embeddings: 35772


In [11]:
print('Build model...')

## implement model here

model = Sequential()

## to use the glove embeddings, your embedding layer would take the vocab size as input dimension, 
## Glove embedding dimension as the output dimsion
## and you will provide the  embedding dictionary as the 'weights' parameter (!important) to the embedding layer.
model.add(Embedding(nb_words, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH,weights=[g_word_embedding_matrix]))
model.add(LSTM(128,recurrent_dropout=0.2,dropout=0.2))
model.add(Dense(128,activation='relu'))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
## compille it here according to instructions

model.summary()

print("Model Built")

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 250, 300)          37275900  
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               219648    
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 258       
Total params: 37,512,318
Trainable params: 37,512,318
Non-trainable params: 0
_________________________________________________________________
Model Built


In [10]:
print('Train...')
model.fit(x_train, y_train,
          batch_size=32,
          epochs=5,
          validation_data=(x_val, y_val),
          verbose = 1,
         shuffle = True)

Train...
Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 428s 21ms/step - loss: 0.4288 - acc: 0.7930 - val_loss: 0.2818 - val_acc: 0.8836
Epoch 2/5
20000/20000 [==============================] - 426s 21ms/step - loss: 0.2064 - acc: 0.9197 - val_loss: 0.2599 - val_acc: 0.8944
Epoch 3/5
20000/20000 [==============================] - 424s 21ms/step - loss: 0.1060 - acc: 0.9613 - val_loss: 0.2963 - val_acc: 0.8892
Epoch 4/5
20000/20000 [==============================] - 424s 21ms/step - loss: 0.0511 - acc: 0.9823 - val_loss: 0.3819 - val_acc: 0.8922
Epoch 5/5
20000/20000 [==============================] - 420s 21ms/step - loss: 0.0263 - acc: 0.9914 - val_loss: 0.4924 - val_acc: 0.8858


# Dont attempt this

Stacking LSTM layers

Unfortunately it takes very long to train, be aware we can stack LTMSs over each other like this.
This requires bottom LSTM to return a sequences instead instead of single vector, which becomes input for the top LSTM.


![title](img/model5.jpg)

# Part E

Using Convolutional Networks (10 points)

Construct the model, shown below. Use the same loss functions and optimizers as before

Correction: The Embedding Layer Dimension (1st box) is 300, not 128.

![title](img/model6.jpg)

In [10]:
print('Build model...')

## implement model here

from keras.layers import Conv1D
model = Sequential()
model.add(Embedding(nb_words, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH,weights=[g_word_embedding_matrix]))
model.add(Conv1D(128, 3, activation='relu'))
model.add(Dropout(0.2))
model.add(Conv1D(64, 3, activation='relu'))
model.add(Dropout(0.2))
model.add(Conv1D(32, 3, activation='relu'))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
## compille it here according to instructions

model.summary()

print("Model Built")

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 250, 300)          37275900  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 248, 128)          115328    
_________________________________________________________________
dropout_1 (Dropout)          (None, 248, 128)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 246, 64)           24640     
_________________________________________________________________
dropout_2 (Dropout)          (None, 246, 64)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 244, 32)           6176      
_________________________________________________________________
dropout_3 (Dropout)          (None, 244, 32)           0     

In [13]:

print('Train...')
model.fit(x_train, y_train,
          batch_size=32,
          epochs=5,
          validation_data=(x_val, y_val),
          verbose = 1,
         shuffle = True)

Train...
Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 251s 13ms/step - loss: 0.4194 - acc: 0.7896 - val_loss: 0.3015 - val_acc: 0.8680
Epoch 2/5
20000/20000 [==============================] - 248s 12ms/step - loss: 0.2089 - acc: 0.9196 - val_loss: 0.2634 - val_acc: 0.8998
Epoch 3/5
20000/20000 [==============================] - 249s 12ms/step - loss: 0.1206 - acc: 0.9555 - val_loss: 0.3417 - val_acc: 0.8740
Epoch 4/5
20000/20000 [==============================] - 248s 12ms/step - loss: 0.0691 - acc: 0.9752 - val_loss: 0.4041 - val_acc: 0.8762
Epoch 5/5
20000/20000 [==============================] - 249s 12ms/step - loss: 0.0410 - acc: 0.9855 - val_loss: 0.4349 - val_acc: 0.8898


# Part F

Model constructed : (5 points)

Test Accuracy Over 87.5%: (5 Points)

Bonus: Min(10, Square of (test_score - 88%))

Create your best model, use Validation score to judge your best model and check accuracy on test set


In [11]:
print('Build model...')

## implement model here
from keras.layers import Conv1D

model_1 = Sequential()
model_1.add(Embedding(nb_words, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH,weights=[g_word_embedding_matrix]))
model_1.add(Conv1D(128, 3, activation='relu'))
model_1.add(Dropout(0.2))
model_1.add(Conv1D(64, 3, activation='relu'))
model_1.add(Dropout(0.2))
# model_1.add(Conv1D(32, 3, activation='relu'))
# model_1.add(Dropout(0.2))
model_1.add(LSTM(8))
#model.add(Flatten())
model_1.add(Dense(256,activation='relu'))
model_1.add(Dense(2, activation='softmax'))
model_1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
## compille it here according to instructions

model_1.summary()

print("Model Built")

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 250, 300)          37275900  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 248, 128)          115328    
_________________________________________________________________
dropout_1 (Dropout)          (None, 248, 128)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 246, 64)           24640     
_________________________________________________________________
dropout_2 (Dropout)          (None, 246, 64)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 8)                 2336      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               2304  

You can keep saving models with different names in model_name, 

so you can retrieve their weights again for testing, you dont have to retrain 
(You would have to initialize the model definition again).

In [12]:
wt_dir = "./weights/"
model_name = 'model_best'
early_stopping =EarlyStopping(monitor='val_acc', patience=2)
bst_model_path = wt_dir + model_name + '.h5'
model_checkpoint = ModelCheckpoint(bst_model_path, monitor='val_acc', save_best_only=True, save_weights_only=True)

print('Train...')
model_1.fit(x_train, y_train,
          batch_size=32,
          epochs=7,
          validation_data=(x_val, y_val),
          verbose = 1,
         shuffle = True,
         callbacks=[early_stopping, model_checkpoint])



Train...
Train on 20000 samples, validate on 5000 samples
Epoch 1/7
20000/20000 [==============================] - 340s 17ms/step - loss: 0.5290 - acc: 0.7364 - val_loss: 0.3709 - val_acc: 0.8344
Epoch 2/7
20000/20000 [==============================] - 341s 17ms/step - loss: 0.3460 - acc: 0.8560 - val_loss: 0.2995 - val_acc: 0.8784
Epoch 3/7
20000/20000 [==============================] - 342s 17ms/step - loss: 0.2458 - acc: 0.9048 - val_loss: 0.3003 - val_acc: 0.8770
Epoch 4/7
20000/20000 [==============================] - 340s 17ms/step - loss: 0.1947 - acc: 0.9274 - val_loss: 0.3079 - val_acc: 0.8704


If you plan on using Ensemble averaging, feel free to edit the code below or add multiple models.

Make sure they get saved and can be retrieved when executing serially.

In [14]:
model_1.load_weights(bst_model_path)
scores = model_1.evaluate(x_test, y_test, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

25000/25000 [==============================] - 54s 2ms/step
Accuracy: 87.90%


# Part G

Explain how Dense, LSTM and Convolution Layers work.

Explain Relu, Dropout, and Softmax work.

Analyze the architectures you constructed, with the accuracies you achieved and the training time it took. 

What are some insights you gained with these experiments? 

(5 Points)


Dense Layers: changes the dimensions of the vector. They are fully connected, which means the nodes in one layer are connected with nodes in previous layer.
LSTM:  a special kind of RNN, which can learn order dependence in sequence prediction problems. It is used to build a block of hidden layers for an eventually bigger recurrent neural network

Convolution Layers:  are not fully connected on the input layer. This layer creates a convolution kernel that is convolved with the layer input over a single spatial (or temporal) dimension to produce a tensor of outputs. 

Relu: an activation function defined as f(x)=max(0,x)

Droput: is a regularization technique for neural network models.Dropout is a technique where randomly selected neurons are ignored during training. They are “dropped-out” randomly. This means that their contribution to the activation of downstream neurons is temporally removed on the forward pass and any weight updates are not applied to the neuron on the backward pass.

Softmax: a generalization of logistic regression to the case where we want to handle multiple classes

I used pretrained embedding layer, 2 convolution layers with dropout and one LTSM layer. The test accury is 87.90%. It its a little bit more time to train the model as it has more layers.

Pretrained embedding, LSTM and dropout are able to significantly imporve the performance. However, if I built too many layers or set the parameter too large, it will cause overfitting problem. For example, I first built a model with 3 convolution layers and LTSM(128), it achieved high training accuracy but only about 50% of test accuracy. 